# Cria simulação (Geração de amostras Sintéticas)

In [19]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Configuração da semente para reprodutibilidade
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# Definir a arquitetura da GAN
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Função para treinar a GAN
def train_gan(real_data, num_features, latent_dim, num_samples, num_epochs=5000, batch_size=32):
    # Normalizar os dados reais
    scaler = MinMaxScaler()
    try:
        real_data = scaler.fit_transform(real_data)
    except ValueError as e:
        raise ValueError(f"Erro na normalização dos dados reais: {e}")

    # Converter para tensores
    real_data = torch.tensor(real_data, dtype=torch.float32)

    # Inicializar o gerador e o discriminador
    generator = Generator(latent_dim, num_features)
    discriminator = Discriminator(num_features)

    # Definir otimizadores e função de perda
    criterion = nn.BCELoss()
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

    for epoch in range(num_epochs):
        # Treinar o discriminador
        for _ in range(max(1, real_data.size(0) // batch_size)):
            idx = np.random.randint(0, real_data.size(0), batch_size)
            real_samples = real_data[idx]

            # Labels reais e falsos
            real_labels = torch.ones((real_samples.size(0), 1))
            fake_labels = torch.zeros((real_samples.size(0), 1))

            # Amostras do gerador
            noise = torch.randn((real_samples.size(0), latent_dim))
            fake_samples = generator(noise)

            # Predições do discriminador
            real_preds = discriminator(real_samples)
            fake_preds = discriminator(fake_samples.detach())

            # Perda do discriminador
            loss_real = criterion(real_preds, real_labels)
            loss_fake = criterion(fake_preds, fake_labels)
            loss_D = loss_real + loss_fake

            optimizer_D.zero_grad()
            loss_D.backward()
            optimizer_D.step()

        # Treinar o gerador
        noise = torch.randn((batch_size, latent_dim))
        fake_samples = generator(noise)
        fake_preds = discriminator(fake_samples)
        loss_G = criterion(fake_preds, torch.ones((batch_size, 1)))

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        # Exibir progresso
        if epoch % 500 == 0 or epoch == num_epochs - 1:
            print(f"Epoch [{epoch}/{num_epochs}] - Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

    # Gerar novas amostras sintéticas
    noise = torch.randn((num_samples, latent_dim))
    synthetic_data = generator(noise).detach().numpy()

    # Reverter a normalização
    synthetic_data = scaler.inverse_transform(synthetic_data)
    return synthetic_data

# Função principal para processar múltiplos datasets
def generate_synthetic_samples(input_directory, output_directory, total_samples=2000):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Carregar datasets
    datasets = {}
    for file in os.listdir(input_directory):
        filepath = os.path.join(input_directory, file)
        filename, ext = os.path.splitext(file)
        if ext == '.txt':
            try:
                print(f"Lendo dataset: {file}")
                df = pd.read_csv(filepath, delimiter=',')
                df = df.apply(pd.to_numeric, errors='coerce')  # Forçar conversão para numérico
                df = df.dropna()  # Remover linhas com valores não numéricos
                if not df.empty:
                    datasets[filename] = df
                else:
                    print(f"{file} está vazio após limpeza.")
            except Exception as e:
                print(f"Erro ao carregar {file}: {e}")

    if not datasets:
        print("Nenhum dataset válido encontrado. Encerrando...")
        return

    # Gerar amostras sintéticas para cada dataset
    for name, df in datasets.items():
        num_features = df.shape[1]
        samples_per_dataset = max(1, total_samples // len(datasets))
        latent_dim = 10  # Dimensão do espaço latente

        print(f"Gerando {samples_per_dataset} amostras sintéticas para {name}...")
        try:
            synthetic_data = train_gan(df.values, num_features, latent_dim, samples_per_dataset)

            # Verificar se já existe um arquivo de saída
            output_path = os.path.join(output_directory, f"{name}.txt")
            if os.path.exists(output_path):
                existing_data = pd.read_csv(output_path, delimiter=',')
                existing_data['source'] = 'original'
            else:
                existing_data = df.copy()
                existing_data['source'] = 'original'

            synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)
            synthetic_df['source'] = 'synthetic'

            # Combinar os dados originais e sintéticos
            combined_data = pd.concat([existing_data, synthetic_df], ignore_index=True)

            # Normalizar os dados combinados
            scaler = MinMaxScaler()
            normalized_data = scaler.fit_transform(combined_data.drop(columns=['source']))
            normalized_df = pd.DataFrame(normalized_data, columns=combined_data.columns[:-1])
            normalized_df['source'] = combined_data['source'].values

            # Salvar os dados normalizados
            normalized_df.to_csv(output_path, sep=',', index=False)
            print(f"Amostras normalizadas salvas em {output_path}")
        except Exception as e:
            print(f"Erro ao gerar amostras para {name}: {e}")

# Diretórios de entrada e saída
input_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\1-tratamento-target-encoding-normalizado"
output_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\1-tratamento"

# Gerar amostras sintéticas
generate_synthetic_samples(input_directory, output_directory)


Lendo dataset: tratamento_china.txt
Lendo dataset: tratamento_cocomo81.txt
Lendo dataset: tratamento_desharnais.txt
Lendo dataset: tratamento_maxwell.txt
Gerando 500 amostras sintéticas para tratamento_china...
Epoch [0/5000] - Loss D: 1.3442, Loss G: 0.6790
Epoch [500/5000] - Loss D: 0.3864, Loss G: 2.2634
Epoch [1000/5000] - Loss D: 0.3813, Loss G: 2.5238
Epoch [1500/5000] - Loss D: 0.2655, Loss G: 3.2843
Epoch [2000/5000] - Loss D: 0.3484, Loss G: 3.4396
Epoch [2500/5000] - Loss D: 0.3255, Loss G: 2.7824
Epoch [3000/5000] - Loss D: 0.1266, Loss G: 3.8165
Epoch [3500/5000] - Loss D: 0.1032, Loss G: 4.1187
Epoch [4000/5000] - Loss D: 0.0957, Loss G: 4.3513
Epoch [4500/5000] - Loss D: 0.0801, Loss G: 3.7662
Epoch [4999/5000] - Loss D: 0.2042, Loss G: 4.0304
Amostras normalizadas salvas em C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao\1-tratamento\tratamento_china.txt
Gerando 500 amostras sintéticas para tratamento_co

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Configuração da semente para reprodutibilidade
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# Definir a arquitetura da GAN
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Função para treinar a GAN
def train_gan(real_data, num_features, latent_dim, num_samples, num_epochs=5000, batch_size=32):
    scaler = MinMaxScaler()
    real_data = scaler.fit_transform(real_data)
    real_data = torch.tensor(real_data, dtype=torch.float32)

    generator = Generator(latent_dim, num_features)
    discriminator = Discriminator(num_features)

    criterion = nn.BCELoss()
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

    for epoch in range(num_epochs):
        for _ in range(max(1, real_data.size(0) // batch_size)):
            idx = np.random.randint(0, real_data.size(0), batch_size)
            real_samples = real_data[idx]

            real_labels = torch.ones((real_samples.size(0), 1))
            fake_labels = torch.zeros((real_samples.size(0), 1))

            noise = torch.randn((real_samples.size(0), latent_dim))
            fake_samples = generator(noise)

            real_preds = discriminator(real_samples)
            fake_preds = discriminator(fake_samples.detach())

            loss_real = criterion(real_preds, real_labels)
            loss_fake = criterion(fake_preds, fake_labels)
            loss_D = loss_real + loss_fake

            optimizer_D.zero_grad()
            loss_D.backward()
            optimizer_D.step()

        noise = torch.randn((batch_size, latent_dim))
        fake_samples = generator(noise)
        fake_preds = discriminator(fake_samples)
        loss_G = criterion(fake_preds, torch.ones((batch_size, 1)))

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        if epoch % 500 == 0 or epoch == num_epochs - 1:
            print(f"Epoch [{epoch}/{num_epochs}] - Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

    noise = torch.randn((num_samples, latent_dim))
    synthetic_data = generator(noise).detach().numpy()
    synthetic_data = scaler.inverse_transform(synthetic_data)
    return synthetic_data

# Função para processar subpastas e gerar saídas
def process_subfolders(input_directory, output_directory, total_samples=2000):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            filepath = os.path.join(root, file)
            relative_path = os.path.relpath(root, input_directory)
            output_subdir = os.path.join(output_directory, relative_path)
            os.makedirs(output_subdir, exist_ok=True)

            if file.endswith('.txt'):
                try:
                    print(f"Lendo arquivo: {filepath}")
                    df = pd.read_csv(filepath, delimiter=',')
                    df = df.apply(pd.to_numeric, errors='coerce').dropna()
                    if df.empty:
                        print(f"Arquivo {file} está vazio após a limpeza.")
                        continue

                    num_features = df.shape[1]
                    samples_per_dataset = total_samples // len(files)
                    synthetic_data = train_gan(df.values, num_features, latent_dim=10, num_samples=samples_per_dataset)

                    output_path = os.path.join(output_subdir, file)
                    synthetic_df = pd.DataFrame(synthetic_data, columns=df.columns)
                    synthetic_df.to_csv(output_path, sep=',', index=False)
                    print(f"Arquivo gerado: {output_path}")

                except Exception as e:
                    print(f"Erro ao processar {file}: {e}")

# Diretórios de entrada e saída
input_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\2-geracao-variaveis"
output_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\3-simulacao"

# Processar subpastas e gerar saídas
process_subfolders(input_directory, output_directory)
